In [ ]:
#importing files
import pandas as pd
from urllib import request
import json
from urllib.error import HTTPError


arrived_cities = ['Vancouver','Montreal','Halifax','Calgary','Toronto']
AWR_ports = ['Vancouver','Montreal','Halifax']
MLB_arrival = ['Vancouver','Montreal','Halifax', 'Prince Rupert']
result = []
error = []


base_api_url = "https://api.maersk.com/track/"
# input your own list of containers
container_num = container_list

for z in container_num:
  request_url = base_api_url + z
  


  api_request = request.Request(request_url, headers={"Accept": "application/json"})

  try:
    request.urlopen(api_request)
  except HTTPError as err:
    if err.code == 404:
      print("yeo")
      next
  else:
      api_response = request.urlopen(api_request)


      parsed_response = json.loads(api_response.read().decode("utf-8"))





      container = parsed_response['containers']

      try:
      
        #if container is AWR
        if parsed_response['destination']['city'] in AWR_ports:
          for container in parsed_response["containers"]:
            for i in reversed(range (len(container["locations"]))):
              for event_item in container["locations"][i]['events']:
                for j in reversed(range (len(container["locations"][i]['events']))): 

                  # if container is rail loaded
                  if container["locations"][i]['events'][j]['is_current'] == True and container["locations"][i]['city'] in arrived_cities and container["locations"][i]['events'][j]['activity'] == 'GATE-OUT':

                    #finding discharge date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in AWR_ports and container["locations"][a]['events'][b]['activity'] == 'DISCHARG':
                              discharge = container["locations"][a]['events'][b]['actual_time']         
                    result.append({
                        'Container': z,
                        'ETA_to_Port': container['eta_final_delivery'],         
                        'Discharged_from_vessel': discharge,            
                        'Rail_Loaded':container["locations"][i]['events'][j]['actual_time'],
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'AWR',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })


                  # if container is discharged
                  elif container["locations"][i]['events'][j]['is_current'] == True and container["locations"][i]['city'] in arrived_cities and container["locations"][i]['events'][j]['activity'] == 'DISCHARG':
                    result.append({
                        'Container': z,
                        'ETA_to_Port': container['eta_final_delivery'],
                        'Discharged_from_vessel': container["locations"][i]['events'][j]['actual_time'],
                        'Rail_Loaded':'',
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'AWR',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                    })
                    

                  # if current status not in arrived cities = on the water
                  elif container["locations"][i]['events'][j]['is_current'] == True and container["locations"][i]['city'] not in arrived_cities:
                    result.append({
                        'Container': z,
                        'ETA_to_Port': container['eta_final_delivery'],
                        'Discharged_from_vessel': '',
                        'Rail_Loaded':'',
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'AWR',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']

                    })

        #container is MLB
        else:
          for container in parsed_response["containers"]:
            for i in reversed(range (len(container["locations"]))):
              for event_item in container["locations"][i]['events']:
                for j in reversed(range (len(container["locations"][i]['events']))): 


                  # 1) if dray carrier gate out container
                  if container["locations"][i]['events'][j]['is_current'] == True and container["locations"][i]['city'] in parsed_response['destination']['city']  and container["locations"][i]['events'][j]['activity'] == 'GATE-OUT' :

                    #finding rail arrival
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] == parsed_response['destination']['city']  and container["locations"][a]['events'][b]['activity'] == 'OFF-RAIL' and 'actual_time' in container["locations"][a]['events'][b]:
                              off_rail = container["locations"][a]['events'][b]['actual_time']         

                    #finding  rail loaded date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'ON-RAIL':
                              on_rail = container["locations"][a]['events'][b]['actual_time'] 
                    #finding discharge date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'DISCHARG':
                              discharge = container["locations"][a]['events'][b]['actual_time']

                    result.append({
                        'Container': z,
                        'ETA_to_Port': discharge,         
                        'Discharged_from_vessel': discharge,            
                        'Rail_Loaded':on_rail,
                        'Rail_Arrived':off_rail,
                        'Picked_Up':container["locations"][i]['events'][j]['actual_time'],
                        'MLB/AWR':'MLB',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })
                    break

                    
                  # 2) if container arrived inland, dray carrier did not pickup
                  elif container["locations"][i]['events'][j]['is_current'] == True and container["locations"][i]['city'] in parsed_response['destination']['city']  and container["locations"][i]['events'][j]['activity'] == 'OFF-RAIL':
        
                    #finding  rail loaded date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'ON-RAIL':
                              on_rail = container["locations"][a]['events'][b]['actual_time']  


                    #finding discharge date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'DISCHARG':
                              discharge = container["locations"][a]['events'][b]['actual_time']

                    result.append({
                        'Container': z,
                        'ETA_to_Port': discharge,         
                        'Discharged_from_vessel': discharge,            
                        'Rail_Loaded':on_rail,
                        'Rail_Arrived':container["locations"][i]['events'][j]['actual_time'],
                        'Picked_Up':'',
                        'MLB/AWR':'MLB',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })
                    break               
                    
                  # 3) container only just rail loaded
                  elif container["locations"][i]['events'][j]['is_current'] == True  and container["locations"][i]['events'][j]['activity'] == 'ON-RAIL':

                    #finding discharge date
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'DISCHARG':
                              discharge = container["locations"][a]['events'][b]['actual_time']

                    result.append({
                        'Container': z,
                        'ETA_to_Port': discharge,         
                        'Discharged_from_vessel': discharge,            
                        'Rail_Loaded':container["locations"][i]['events'][j]['actual_time'],
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'MLB',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })
                    break
                    
                  # 4) container is discharged at port
                  elif container["locations"][i]['events'][j]['is_current'] == True  and container["locations"][i]['events'][j]['activity'] == 'DISCHARG':      
                    result.append({
                        'Container': z,
                        'ETA_to_Port': container["locations"][i]['events'][j]['actual_time'],         
                        'Discharged_from_vessel': container["locations"][i]['events'][j]['actual_time'],            
                        'Rail_Loaded':'',
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'MLB',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })
                    break

                  # 5) container is on the water 
                  else:
                    #finding eta to port
                    for container in parsed_response["containers"]:
                      for a in reversed(range (len(container["locations"]))):
                        for event_item in container["locations"][a]['events']:
                          for b in reversed(range (len(container["locations"][a]['events']))): 
                            if container["locations"][a]['city'] in MLB_arrival and container["locations"][a]['events'][b]['activity'] == 'DISCHARG':
                              eta = container["locations"][a]['events'][b]['expected_time']

                    result.append({
                        'Container': z,
                        'ETA_to_Port': eta,         
                        'Discharged_from_vessel': '',            
                        'Rail_Loaded':'',
                        'Rail_Arrived':'',
                        'Picked_Up':'',
                        'MLB/AWR':'MLB',
                        'Starting_Location':parsed_response['origin']['country'],
                        'Final_Location':parsed_response['destination']['city']
                        })
      except: 
        error.append({'Container': z})
        next
      
      else: 
        continue
                    

              








In [ ]:
df_result = pd.DataFrame(result)
#df_result.drop_duplicates(subset = ['Container'], keep = 'first',inplace = True)
df_result.iloc[:8].astype(str)

#date formating

df_result['ETA_to_Port'] = pd.to_datetime(df_result['ETA_to_Port'], format = '%Y-%m-%d')
df_result['Discharged_from_vessel'] = pd.to_datetime(df_result['Discharged_from_vessel'], format = '%Y-%m-%d', errors = 'coerce')
df_result['Rail_Loaded'] = pd.to_datetime(df_result['Rail_Loaded'], format = '%Y-%m-%d', errors = 'coerce')
df_result['Rail_Arrived'] = pd.to_datetime(df_result['Rail_Arrived'], format = '%Y-%m-%d', errors = 'coerce')
df_result['Picked_Up'] = pd.to_datetime(df_result['Picked_Up'], format = '%Y-%m-%d', errors = 'coerce')

df_result.sort_values(by=['Picked_Up','Rail_Arrived','Rail_Loaded', 'Discharged_from_vessel'], ascending = False, inplace = True)
df_result.drop_duplicates(subset = ['Container'], keep = 'first',inplace = True)



In [ ]:
from google.colab import files

df_result.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')